In [1]:
# 隨機十篇id
# M.1127219109.A.837
# M.1127227032.A.F3B
# M.1127365129.A.D61
# M.1127395683.A.DB9
# M.1127526873.A.CD5
# M.1127533364.A.C32
# M.1127544042.A.E06
# M.1127586620.A.EDB
# M.1127588070.A.525
# M.1127590679.A.006


In [2]:
# 用比較快快的json parser : ujson
try:
    import ujson as json
except ImportError:
    try:
        import simplejson as json
    except ImportError:
        import json

import math
import numpy as np
from numba import jit


# load tfidf

def readJson2Data(filename):
  f = open(filename, "r", encoding='UTF-8')
  docText = f.read()
  f.close()
  return json.loads(docText)


In [ ]:
SAMPLE = 'sample1000_gossiping_dataset_doc_vector.json'
ALLDATA = 'gossiping_dataset_doc_vector.json'
# data_tfidf = readJson2Data(ALLDATA)





DOC_NAME = 'data_user-docIDs.json'
name_docid = readJson2Data(DOC_NAME)
NLVECTOR = 'name-docID_NLvector.json'
docID_name_NLvector = readJson2Data(NLVECTOR)

In [ ]:
# # tools
# def saveSmallData(data):
#     jsObj = json.dumps(data)
    
#     fileObject = open('sample1000.json', 'w')
#     fileObject.write(jsObj)
#     fileObject.close()  
# # saveSmallData(sample)    

In [ ]:


# sample = {}
# counter = 0
# for k in data_tfidf.keys():
#     if counter == 10:
#         break
#     print(k)
#     data_tfidf[k]
#     sample[k] = data_tfidf[k]

#     counter = counter + 1

In [ ]:

def calNormalizedLength(id):
    vector = {}
    sum = 0
    for word in data_tfidf[id].keys(): 
        sum = sum + math.pow(data_tfidf[id][word], 2)
    
    sqrt = math.sqrt(sum)

    # 每項除除平方根
    for word in data_tfidf[id].keys():
        vector[word] = data_tfidf[id][word] / sqrt
    return vector


In [ ]:

def findCommonWordSet(ida,idb):
    CommonWordSet = set()
    Aset = set( docID_name_NLvector[ida].keys() )
    Bset = set( docID_name_NLvector[idb].keys() )

    for token in Aset:
        if token in Bset:
            CommonWordSet.add(token)
    return CommonWordSet

In [ ]:

def getNLvector(ida,idb):
    return [docID_name_NLvector[ida], docID_name_NLvector[idb]]

In [ ]:
# @jit
def cos(ida,idb):
    # get calNormalizedLength
    # ida_NL = docID_name_NLvector[ida]
    # idb_NL = docID_name_NLvector[idb]
    ida_NL, idb_NL = getNLvector(ida,idb)

    # findCommonWordSet
    commonWordSet = findCommonWordSet(ida,idb)

    # v1 = []
    # v2 = []
    # for word in commonWordSet:
    #     v1.append(ida_NL[word])
    #     v2.append(idb_NL[word])
    
    # # use numpy
    # v1 = np.array(v1)
    # v2 = np.array(v2)
    
    # calCos  內積

    # dot = np.dot(v1,v2)
    sum = 0
    for word in commonWordSet:
        sum = sum + ida_NL[word] * idb_NL[word]
    
    return sum

In [ ]:
cos('M.1127219109.A.837','M.1127227032.A.F3B')

In [ ]:
def isUserValid(name):
    try:
        name_docid[name]
        return True
    except:
        return False


In [ ]:
def ui():
    name = input("請輸入使用者ID")
    if isUserValid(name) == False:
        print("使用者ID不合法或未在資料庫內 請重新輸入")
        return 0
    else:
        return name
        


In [ ]:

results = {}
docIDs = docID_name_NLvector.keys()
taskmap = {}
# map<taskNumber, [docId,Pttdocid]> 
# results<pttdocID,list[result]>


# map<int,docid>

In [ ]:
#開啟多個執行緒，同時執行任務，有幾個執行緒就執行幾個任務
import threading
import time
import queue

class MyThread(threading.Thread):
    def __init__(self, func):
        threading.Thread.__init__(self)
        self.func = func
    def run(self):
        self.func()


def worker():
    while not q.empty():
        item = q.get()  # 或得任務
        print('Processing : ',item)
        doc , pttdoc = taskmap[item]
        # 如果是自己的文章比自己的文章
        if doc == pttdoc:
            continue
        try:
            if type(results[pttdoc]) is list:
                results[pttdoc].append(cos(doc,pttdoc))
            else:
                results[ pttdocdict[pttdocnum] ] = []
                results[ pttdocdict[pttdocnum] ].append(cos(doc,pttdoc))
        except:
            print('!!!error ' + doc + '  '+ pttdoc)



def main(name):
    threads = []
    targetDoclist = name_docid[name]
    # 產生task list
    counter = 0
    for doc in targetDoclist:
        for pttdoc in docIDs:
            taskmap[counter] = []
            taskmap[counter].append(doc)
            taskmap[counter].append(pttdoc)
            q.put(counter)
            counter = counter + 1
        print( q.qsize() ) 

    for i in range(threadNum):   #開啟n個執行緒
        thread = MyThread(worker)
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == '__main__':
    q = queue.Queue()
    threadNum = 24
    
    # name = ui()
    name = 'Kay731'
    if name == 0:
        print("retry")
    else:
        main(name)

In [ ]:
len(result)

In [ ]:
cos('M.1353503637.A.14B','M.1173516369.A.2C4')
cos('M.1353503637.A.14B','M.1173625796.A.F2A')
cos('M.1353503637.A.14B','M.1173629372.A.C93')

In [ ]:
# data_tfidf['M.1353503637.A.14B']
# docID_name['M.1353503637.A.14B']

In [ ]:
@jit(nopython=True)
def testf():
    for i in range(0,550000000):
        if (i%1000000) ==0:
            print(i)

testf()